In [8]:
from IPython.display import display, HTML
import json
import os
import numpy
from tqdm import tqdm
import warnings
import pandas as pd
import json

In [11]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("medicalai/ClinicalBERT")
model = AutoModelForMaskedLM.from_pretrained("medicalai/ClinicalBERT")

ModuleNotFoundError: No module named 'torch'

In [ ]:
def show_query(query):
    """HTML print format for the searched query"""
    return HTML('<br/><div style="font-family: Times New Roman; font-size: 20px;'
                'padding-bottom:12px"><b>Query</b>: '+query+'</div>')

def show_document(idx, doc):
    """HTML print format for document fields"""
    have_body_text = 'body_text' in json.loads(doc.raw)
    body_text = ' Full text available.' if have_body_text else ''
    return HTML('<div style="font-family: Times New Roman; font-size: 18px; padding-bottom:10px">' + 
               f'<b>Document {idx}:</b> {doc.docid} ({doc.score:1.2f}) -- ' +
               f'{doc.lucene_document.get("authors")} et al. ' +
             # f'{doc.lucene_document.get("journal")}. ' +
             # f'{doc.lucene_document.get("publish_time")}. ' +
               f'{doc.lucene_document.get("title")}. ' +
               f'<a href="https://doi.org/{doc.lucene_document.get("doi")}">{doc.lucene_document.get("doi")}</a>.'
               + f'{body_text}</div>')

def show_query_results(query, searcher, top_k=10):
    """HTML print format for the searched query"""
    hits = searcher.search(query)
    display(show_query(query))
    for i, hit in enumerate(hits[:top_k]):
        display(show_document(i+1, hit))
    return hits[:top_k]   

In [ ]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-1.21.0-openjdk-amd64"

In [ ]:
# CHeck java version
!java -version

openjdk version "21.0.8" 2025-07-15
OpenJDK Runtime Environment (build 21.0.8+9-Ubuntu-0ubuntu124.04.1)
OpenJDK 64-Bit Server VM (build 21.0.8+9-Ubuntu-0ubuntu124.04.1, mixed mode, sharing)


In [ ]:
from pyserini.search import pysearch

searcher = pysearch.SimpleSearcher('path/to/index')


JavaException: JVM exception occurred: java.lang.IllegalArgumentException: path/to/index does not exist or is not a directory.
java.lang.IllegalArgumentException: path/to/index does not exist or is not a directory.
	io.anserini.search.SimpleSearcher.<init>(SimpleSearcher.java:126)
	io.anserini.search.SimpleSearcher.<init>(SimpleSearcher.java:119)